## COMP8240 - Major project

In order to show that the core claims made by the original paper still holds
against the unseen dataset, I’m going to conduct new experiment of sentiment
classification by applying DistilBERT model on new custom dataset collected
from Twitter.

###Pre-configuration
Connect to Google Drive, Import libraries, and Authenticate to Twitter etc.

In [ ]:
# Connect to Google Drive
from google.colab import drive 
drive.mount('/content/drive')
dataset_directory = '/content/drive/MyDrive/COMP8240/DistilBERT/'  # change to your own directory

Mounted at /content/drive


Import libraries and packages 

In [ ]:
import tweepy
import json
import pandas as pd
import numpy as np
import re

# Vader installation
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Install transformers
!pip install transformers

import os
import transformers
from transformers import AutoTokenizer
from tokenizers import BertWordPieceTokenizer
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 32.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 33.1 MB/s 
     |████████████████████████████████| 163 kB 67.0 MB/s 
     |████████████████████████████████| 7.6 MB 47.0 MB/s 


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 13.7 MB/s 
     |████████████████████████████████| 212 kB 67.1 MB/s 
     |████████████████████████████████| 115 kB 68.6 MB/s 
     |████████████████████████████████| 127 kB 62.6 MB/s 
     |████████████████████████████████| 115 kB 63.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Initialise Twitter API Credentials, Authentication, and API object

In [ ]:
# Twitter API credentials
api_key = "VsdtFWvxr3c2U8sCMRgf1QQLE"
api_secrets = "4jXJLkRP4dMEb9NIZYdrN422Y71hfj7AiV5l1LPesEYc0PvkwC"
access_token = "1534023871078158336-nNt5zV0E9yIM4vDbgcILvp3VSsZW6d"
access_secret = "G5CMeON0qfLngAPs1wdNFowk30C4UMBn9DgHzHPJMfXuV"

# Authenticate to Twitter and set access token
authenticate = tweepy.OAuthHandler(api_key,api_secrets)
authenticate.set_access_token(access_token,access_secret)

# Create API object
api = tweepy.API(authenticate, wait_on_rate_limit= True)

### Step 1
Collect 500 recent tweets from 20 different users

In [ ]:
# Initialise top 20 user with most followers
top_20_users = ['@barackobama','@justinbieber','@elonmusk','@katyperry','@rihanna','@cristiano','@taylorswift13','@ladygaga','@narendramodi','@theellenshow','@youtube','@kimkardashian','@selenagomez','@nasa','@cnnbrk','@twitter','@jtimberlake','@billgates','@neymarjr','@britneyspears']

# Ref: https://www.socialtracker.io/toplists/top-25-twitter-users-by-followers/

In [ ]:
print(len(top_20_users))

20


In [ ]:
for user in top_20_users:
  print(user)


In [ ]:
# Retrieve 500 recent tweets from 20 different users and export to .json file

max_tweets = 5
tweets_dict = []

for user in top_20_users:
  screen_name = user
  tweets = api.user_timeline(screen_name = screen_name, count=max_tweets, lang= "en",tweet_mode = "extended")
  for json_tweet in tweets:
      tweets_dict.append(json_tweet._json)

with open(dataset_directory+'top-500-tweets-from-20-users.json', mode = 'w') as file:
        file.write(json.dumps(tweets_dict, indent = 4))

### Step 2
Use the vaderSentiment module to calculate the sentiment of each tweet, and then for each of the two users, calculate the average 'compound' sentiment for all their tweets.

In [ ]:
# Initialise the SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [ ]:
# Read in .json file in the form of JSON object and input into an array to hold sentiment
tweets_sentiment = []

with open(dataset_directory+'top-500-tweets-from-20-users.json', encoding='utf-8', mode = 'r') as file:  
    list_of_dicts = json.load(file)
    for dict in list_of_dicts:
        tweets_sentiment.append({'user_name': str(dict['user']['name']),
                     'text': str(dict['full_text']),
                     })

In [ ]:
# Convert into DataFrame
df_tweets_sentiment = pd.DataFrame(tweets_sentiment)

In [ ]:
df_tweets_sentiment.shape

(100, 2)

In [ ]:
# Create functions to be used for pre-processing of raw data

# Pre-process tweet texts, i.e. remove URL, mentions and ReTweets.
def preprocess_tweet (tweet):
  tweet = re.sub(r"@(\w+)", ' ', tweet)       #clean up @mentions
  tweet = re.sub('@[^\s]+','',tweet)          #clean up hashtags
  tweet = re.sub('https?:\/\/\S+', '', tweet) #clean up URLs
  tweet = re.sub('RT[\s]+', '', tweet)        #clean up RT, retweets
  tweet = re.sub('[^\w\s]','', tweet)         #clean up punctuation
  return tweet

#Ref: https://twitter-data-analysis.blogspot.com/2020/08/twitter-data-analysis-part-3-sentiment.html

In [ ]:
# Run pre-processing of tweet texts
df_tweets_sentiment['text'] = df_tweets_sentiment['text'].apply(preprocess_tweet)

In [ ]:
df_tweets_sentiment.head(20)

,user_name,text
0,Barack Obama,Make a plan to vote at \n\nThen get your fami...
1,Barack Obama,And in Wisconsin there are some great Democrat...
2,Barack Obama,We have the power to forge a different future ...
3,Barack Obama,Im here in Milwaukee for a grassroots event to...
4,Barack Obama,And make a plan to vote at \n\nThen encourage...
5,Justin Bieber,
6,Justin Bieber,1 year of preparation GarenaFreeFire FF5thA...
7,Justin Bieber,Go for the Gold Ladies Cannot wait to watch so...
8,Justin Bieber,2022WeverseCon Countdown DDAY\n\n\nTicket sal...
9,Justin Bieber,The countdown begins


In [ ]:
# Calculate polarity scores
tweets_polarity_scores = []

# Variables used
compound_list = []
positive_list = []
negative_list = []
neutral_list = []

for tweet in range(df_tweets_sentiment['text'].shape[0]):
    compound = analyser.polarity_scores(df_tweets_sentiment['text'][tweet])["compound"]
    positive = analyser.polarity_scores(df_tweets_sentiment['text'][tweet])["pos"]
    neutral = analyser.polarity_scores(df_tweets_sentiment['text'][tweet])["neu"]
    negative = analyser.polarity_scores(df_tweets_sentiment['text'][tweet])["neg"]

    tweets_polarity_scores.append({"Compound": compound,
                       "Positive": positive,
                       "Negative": negative,
                       "Neutral": neutral
                  })

In [ ]:
# Convert polarity scores into DataFrame then merge to tweets DataFrame
tweets_sentiments_score = pd.DataFrame.from_dict(tweets_polarity_scores)
df_tweets_sentiment = df_tweets_sentiment.join(tweets_sentiments_score)

In [ ]:
df_tweets_sentiment.head(20)

,user_name,text,Compound,Positive,Negative,Neutral
0,Barack Obama,Make a plan to vote at \n\nThen get your fami...,0.4767,0.129,0.000,0.871
1,Barack Obama,And in Wisconsin there are some great Democrat...,0.6249,0.124,0.000,0.876
2,Barack Obama,We have the power to forge a different future ...,0.6956,0.123,0.000,0.877
3,Barack Obama,Im here in Milwaukee for a grassroots event to...,0.4019,0.094,0.000,0.906
4,Barack Obama,And make a plan to vote at \n\nThen encourage...,0.7506,0.130,0.000,0.870
5,Justin Bieber,,0.0000,0.000,0.000,0.000
6,Justin Bieber,1 year of preparation GarenaFreeFire FF5thA...,0.0000,0.000,0.000,1.000
7,Justin Bieber,Go for the Gold Ladies Cannot wait to watch so...,0.8478,0.301,0.000,0.699
8,Justin Bieber,2022WeverseCon Countdown DDAY\n\n\nTicket sal...,0.0000,0.000,0.000,1.000
9,Justin Bieber,The countdown begins,0.0000,0.000,0.000,1.000


In [ ]:
df_tweets_sentiment.shape

(100, 6)

In [ ]:
# Remove rows with empty values

# Get indexes where column values equal to 0.0
indexNames = df_tweets_sentiment[(df_tweets_sentiment['Compound'] == 0.0) & (df_tweets_sentiment['Positive'] == 0.0) & (df_tweets_sentiment['Negative'] == 0.0) & (df_tweets_sentiment['Neutral'] == 0.0)].index
# Delete these row indexes from dataFrame
df_tweets_sentiment.drop(indexNames , inplace=True)

In [ ]:
df_tweets_sentiment.head(20)

,user_name,text,Compound,Positive,Negative,Neutral
0,Barack Obama,Make a plan to vote at \n\nThen get your fami...,0.4767,0.129,0.000,0.871
1,Barack Obama,And in Wisconsin there are some great Democrat...,0.6249,0.124,0.000,0.876
2,Barack Obama,We have the power to forge a different future ...,0.6956,0.123,0.000,0.877
3,Barack Obama,Im here in Milwaukee for a grassroots event to...,0.4019,0.094,0.000,0.906
4,Barack Obama,And make a plan to vote at \n\nThen encourage...,0.7506,0.130,0.000,0.870
6,Justin Bieber,1 year of preparation GarenaFreeFire FF5thA...,0.0000,0.000,0.000,1.000
7,Justin Bieber,Go for the Gold Ladies Cannot wait to watch so...,0.8478,0.301,0.000,0.699
8,Justin Bieber,2022WeverseCon Countdown DDAY\n\n\nTicket sal...,0.0000,0.000,0.000,1.000
9,Justin Bieber,The countdown begins,0.0000,0.000,0.000,1.000
10,Elon Musk,What could we do to make it better than TikTok,0.4404,0.244,0.000,0.756


### Step 3.

Label each tweets according to polarity scores.

Label 0 = Neutral

Label 1 = Positive

Label 2 = Negative

*   Positive sentiment: compound score >= 0.05
*   Neutral sentiment: 0.05 > compound score > -0.05
*   Negative sentiment: compound score <= -0.05

(ref: https://github.com/cjhutto/vaderSentiment)

In [ ]:
# Add new columns for sentiment label
df_tweets_sentiment['labels'] = 2      # Default value is Neutral value

# Update 'sentiment' to 1 when compound score is equal to or greater than 0.05. i.e. positive sentiment
df_tweets_sentiment.loc[df_tweets_sentiment['Compound'] >= 0.05, ['labels']] = 1

# Update 'sentiment' to 2 when compound score is equal to or less than -0.05. i.e. negative sentiment
df_tweets_sentiment.loc[df_tweets_sentiment['Compound'] <= -0.05, ['labels']] = 0

In [ ]:
# Remove neutral sentiment
# Get indexes where column values equal to 2
indexNames = df_tweets_sentiment[(df_tweets_sentiment['labels'] == 2)].index
# Delete these row indexes from dataFrame
df_tweets_sentiment.drop(indexNames , inplace=True)

In [ ]:
# Randomly select for preview of dataset

df_tweets_sentiment[["user_name", "text", "labels"]].sample(20)

,user_name,text,labels
14,Elon Musk,He sucks,0
50,YouTube,Im not sure how to do \ncheck YouTube,0
32,Taylor Swift,Bejeweled video is out NOW Directed by this ti...,1
34,Taylor Swift,How did I get this lucky having you guys out h...,1
29,Cristiano Ronaldo,How have I been spending my recovery sessions ...,1
39,Lady Gaga,Today in 2010 24yearold Lady Gaga delivers a ...,0
27,Cristiano Ronaldo,Great team effort and a good victory We stand ...,1
63,Selena Gomez,Then they wont feel alone A cryptic clue from...,1
28,Cristiano Ronaldo,Back on track with the same commitment and ded...,1
74,CNN Breaking News,Nancy Pelosi says her children and grandchildr...,0


In [ ]:
df_tweets_sentiment.shape

(2384, 7)

### Step 4. 
Conduct Sentiment Classification analysis using DistilBERT

In [ ]:
# Check for any NULL
df_tweets_sentiment.isnull().values.any()

False

In [ ]:
# create a Dataset from the data frame and split it into a test and train set
# Ref: https://medium.com/nlplanet/fine-tuning-distilbert-on-senator-tweets-a6f2425ca50e
from datasets import Dataset

# Train set = 80%, test set = 20%
#data_train,data_test = train_test_split(df_tweets_sentiment,test_size=0.2,random_state=1)

# Put clean data in a dataset split into train and test sets
dataset = Dataset.from_pandas(df_tweets_sentiment).train_test_split(train_size=0.8, seed=123)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['user_name', 'text', 'Compound', 'Positive', 'Negative', 'Neutral', 'labels', '__index_level_0__'],
        num_rows: 2567
    })
    test: Dataset({
        features: ['user_name', 'text', 'Compound', 'Positive', 'Negative', 'Neutral', 'labels', '__index_level_0__'],
        num_rows: 642
    })
})


In [ ]:
# cast the labels column as a class-encoded column, which tells the dataset where to find the labels for the data and automatically formats it for training

# Cast labels column as class labels
dataset = dataset.class_encode_column('labels')

Stringifying the column:   0%|          | 0/3 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/3 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Stringifying the column:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# Tokenize dataset
# Load DistilBERT tokenizer and tokenize (encode) the texts
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# make a list of columns to remove from the dataset upon tokenization and tokenize the tweets in the text column

# Make a list of columns to remove before tokenization
cols_to_remove = [col for col in dataset["train"].column_names if col != "labels"]
print(cols_to_remove)

['user_name', 'text', 'Compound', 'Positive', 'Negative', 'Neutral', '__index_level_0__']


In [ ]:
# Tokenize and encode the dataset
def tokenize(batch):
    tokenized_batch = tokenizer(batch['text'], padding=True, truncation=True, max_length=128)
    return tokenized_batch

dataset_enc = dataset.map(tokenize, batched=True, remove_columns=cols_to_remove, num_proc=4)

# Set dataset format for PyTorch
dataset_enc.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# Check the output
print(dataset_enc["train"].column_names)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

['labels', 'input_ids', 'attention_mask']


In [ ]:
# instantiate a DataLoader for each split of the dataset to feed it to the model

from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

# Instantiate a data collator with dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create data loaders for to reshape data for PyTorch model --Original batch size = 8
train_dataloader = DataLoader(
    dataset_enc["train"], shuffle=True, batch_size=16, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    dataset_enc["test"], batch_size=16, collate_fn=data_collator
)

In [ ]:
# Load the model

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForSequenceClassification

# Dynamically set number of class labels based on dataset
num_labels = dataset["train"].features["labels"].num_classes
print(f"Number of labels: {num_labels}")

# Load model from checkpoint
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

Number of labels: 2


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

In [ ]:
# Define hyperparameters, optimizer, and learning rate scheduler

from transformers import AdamW
from transformers import get_scheduler

# Model parameters
learning_rate=2e-5  #learning_rate = 5e-5        # learning_rate=2e-5
num_epochs = 5

# Create the optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Further define learning rate scheduler
num_training_batches = len(train_dataloader)
num_training_steps = num_epochs * num_training_batches
lr_scheduler = get_scheduler(
    "linear",                   # linear decay
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
# move the model to the GPU, since we are using Google Colab

# Set the device automatically (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Move model to device
model.to(device)

cuda


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
# Train model

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

# Train the model with PyTorch training loop
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        

  0%|          | 0/805 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
# Save model to disk
model.save_pretrained(dataset_directory+"/distilbert-tweets-model")

In [ ]:
# Evaluate model

from datasets import load_metric

# Load metric
metric = load_metric("glue", "mrpc")

# Iteratively evaluate the model and compute metrics
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

# Get model accuracy and F1 score
metric.compute()

{'accuracy': 0.9158878504672897, 'f1': 0.9311224489795917}

In [ ]:
# Following docuemnt was used as reference when creating this notebook.
# REF: https://medium.com/nlplanet/fine-tuning-distilbert-on-senator-tweets-a6f2425ca50e